In [1]:
import sys
import os
sys.path.append("/local/scratch/js2173/pytorch/Selectively-Retexuring-Subimages/submodules/PSPNet-Keras-tensorflow") # access submodules
# filter out paths added by other users of server...
sys.path = [p for p in sys.path if not p.startswith('/local/scratch') or p.startswith('/local/scratch/js2173')]

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = '' # set to 0 to use GPU if its available

In [3]:
from os.path import splitext, join
import numpy as np
import argparse
from scipy import misc, ndimage
from keras import backend as K
from keras.models import model_from_json, load_model
import tensorflow as tf
import layers_builder as layers
from python_utils import utils
from python_utils.preprocessing import preprocess_img

from pspnet import PSPNet101


Using TensorFlow backend.


In [12]:
pspnet_path =  '/local/scratch/js2173/pytorch/Selectively-Retexuring-Subimages/submodules/PSPNet-Keras-tensorflow'
filename = "epoch100_real_image.jpg"
input_path = join('example_images', filename)
input_size = 512
output_path = join('example_results', filename)
model = "pspnet101_cityscapes"

In [14]:
sess = tf.Session()
K.set_session(sess)

with sess.as_default():
    img = misc.imread(input_path, mode='RGB')
    cimg = misc.imresize(img, (input_size, input_size))
    pspnet = PSPNet101(nb_classes=19, input_shape=(713, 713), weights_name=model, weights_path=pspnet_path)


    probs = pspnet.predict(img, flip_evaluation=False)

    print("Writing results...")
    cm = np.argmax(probs, axis=2)
    pm = np.max(probs, axis=2)

    color_cm = utils.add_color(cm)
    # color cm is [0.0-1.0] img is [0-255]
    alpha_blended = 0.5 * color_cm * 255 + 0.5 * img
    filename, ext = splitext(output_path)
    misc.imsave(filename + "_seg_read" + ext, cm)
    misc.imsave(filename + "_seg" + ext, color_cm)
    misc.imsave(filename + "_probs" + ext, pm)
    misc.imsave(filename + "_seg_blended" + ext, alpha_blended)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


/local/scratch/js2173/pytorch/Selectively-Retexuring-Subimages/submodules/PSPNet-Keras-tensorflow/weights/keras/pspnet101_cityscapes.h5
Keras model & weights found, loading...
Predicting...
Finished prediction...
Writing results...


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:20: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:21: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


In [13]:
splitext(output_path)

('example_results/epoch100_real_image', '.jpg')

In [ ]:
import sys
sys.executable